<h1 style="text-align:center;"><b>Laboratorio 8</b></h1>
<h3 style="text-align:center;">Alina Carías (22539), Ariela Mishaan (22052), Marcos Díaz (221102) </h3>

In [0]:
#pip install openpyxl

In [0]:
import pandas as pd
import openpyxl
import numpy as np

# Carga de Datos y Análisis Exploratorio

## 1. Cargar todas las hojas de Excel

In [0]:
# Cargar todas las hojas del Excel
file = "/Volumes/workspace/default/lab8-accidentes/datos.xlsx"
xls = pd.ExcelFile(file)

# Ver qué hojas tenemos
print(xls.sheet_names) 

['Portada-Presentación', 'Índice', 'Directorio ', 'Procedimiento', 'cuadro 1', 'cuadro 2', 'cuadro 3', 'cuadro 4', 'cuadro 5', 'cuadro 6', 'cuadro 7', 'cuadro 8', 'cuadro 9', 'cuadro 10', 'cuadro 11', 'cuadro 12', 'cuadro 13', 'cuadro 14', 'cuadro 15', 'cuadro 16', 'cuadro 17', 'cuadro 18', 'cuadro 19', 'cuadro 20', 'cuadro 21', 'cuadro 22', 'cuadro 23', 'cuadro 24', 'cuadro 25', 'cuadro 26', 'cuadro 27', 'cuadro 28', 'cuadro 29', 'cuadro 30', 'cuadro 31', 'cuadro 32', 'cuadro 33', 'cuadro 34', 'cuadro 35', 'cuadro 36', 'cuadro 37', 'cuadro 38', 'cuadro 39', 'cuadro 40', 'cuadro 41', 'cuadro 42', 'cuadro 43', 'cuadro 44', 'cuadro 45', 'cuadro 46', 'cuadro 47', 'cuadro 48', 'cuadro 49', 'cuadro 50', 'cuadro 51', 'cuadro 52', 'cuadro 53', 'cuadro 54', 'cuadro 55', 'cuadro 56', 'cuadro 57', 'cuadro 58', 'cuadro 59', 'cuadro 60', 'cuadro 61', 'cuadro 62', 'cuadro 63', 'cuadro 64', 'cuadro 65', 'Nota técnica', 'Ficha técnica']


In [0]:
# Ejemplo de una hoja

datos = pd.read_excel('/Volumes/workspace/default/lab8-accidentes/datos.xlsx', sheet_name='cuadro 1', index_col=None)
#datos

Se puede ver que el Excel trae 65 hojas con información distinta de los accidentes. Como se puede ver en el índice de este Excel, hay 3 bases principales, cada una con información de distintas categorías: hechos de tránsito, vehículos involucrados, y fallecidos y lesionados. La cantidad de hojas es tan grande porque cada una tiene distinto nivel de detalle y de estratificación. 

## 2. Reformateo de hojas

In [0]:
fila_indice1 = 8 # revisada desde el excel para algunas hojas
dimensiones = ["Departamento", "Mes", "Anio", "Weekday"]

In [0]:
def seleccionar_rango(df, fila_indice, dim_filas, dim_columnas):
    """
    df: DataFrame leído
    fila_indice: índice donde comienza el header
    dim_filas: variable que está en las filas (ej: "Departamento" o "Mes")
    dim_columnas: variable que está en las columnas (ej: "Año", "Mes", "Weekday")
    """
    # Filas: siempre omitir header + posibles totales iniciales
    if dim_filas in ["Departamento", "Mes"]:
        filas = slice(1, 22)  # ajusta según tus hojas, aquí ejemplo general
    else:
        filas = slice(1, df.shape[0])
    
    # Columnas: si la variable de columnas es "Año", selecciona 1:6
    if dim_columnas == "Año":
        columnas = slice(0, 6)
    elif dim_columnas == "Mes":
        columnas = slice(0, 14)
    elif dim_columnas == "Weekday":
        columnas = slice(0, 9)
    else:
        columnas = slice(0, df.shape[1])
    
    return df.iloc[filas, columnas]

def transformar_hoja(file, sheet_name, fila_indice, id_vars_original, var_name, value_name):
    """
    file: ruta del Excel
    sheet_name: nombre de la hoja
    fila_indice: fila donde empieza el header (pandas index)
    id_vars_original: lista de columnas que ya existen y se usarán como id_vars
    var_name: nombre de la columna de variable (melt)
    value_name: nombre de la columna de valores (melt)
    """
    # Leer hoja
    df = pd.read_excel(file, sheet_name=sheet_name, header=fila_indice+1)
    
    # Si id_vars_original es índice numérico, mantener solo esas columnas
    if isinstance(id_vars_original[0], int):
        df = df.iloc[:, id_vars_original + list(range(max(id_vars_original)+1, df.shape[1]))]
    
    # Renombrar primera columna si está como 'Unnamed: 0'
    if 'Unnamed: 0' in df.columns:
        df.rename(columns={'Unnamed: 0': id_vars_original[0]}, inplace=True)

    if 'Unnamed: 1' in df.columns:
        df.drop(df.columns[1], axis=1)

    # Get the name of the first column
    first_col = df.columns[0]

    # Check if 'Total' is present in the first column and drop those rows
    if 'Total' in df[first_col].values:
        df = df[df[first_col] != 'Total']
    
    df = seleccionar_rango(df, fila_indice, id_vars_original[0], var_name)

    # Melt
    df_largo = df.melt(
        id_vars=id_vars_original,
        var_name=var_name,
        value_name=value_name
    )
    
    # Agregar columnas faltantes
    for dim in dimensiones:
        if dim not in df_largo.columns:
            df_largo[dim] = np.nan
    
    # Reordenar columnas: Departamento, Mes, Año, Weekday, variable
    columnas_final = ["Departamento", "Mes", "Anio", "Weekday", value_name]
    df_largo = df_largo[[col for col in columnas_final if col in df_largo.columns]]
    
    return df_largo

In [0]:
hojas_info = {
    "cuadro 1": {"id_vars": ["Departamento"], "var_name":"Anio", "value_name":"accidentes_totales"},

    "cuadro 2": {"id_vars": ["Departamento"], "var_name":"Mes", "value_name":"accidentes_totales"},
    
    "cuadro 3": {"id_vars": ["Departamento"], "var_name":"Weekday", "value_name":"accidentes_totales"},
    
    "cuadro 4": {"id_vars": ["Mes"], "var_name":"Anio", "value_name":"accidentes_totales"},
    
    "cuadro 5": {"id_vars": ["Mes"], "var_name":"Weekday", "value_name":"accidentes_totales"},
    # agregar las demás hojas aquí
}

lista_df = []

for hoja, params in hojas_info.items():
    df_transformado = transformar_hoja(
        file=file,
        sheet_name=hoja,
        fila_indice=fila_indice1,
        id_vars_original=params["id_vars"],
        var_name=params["var_name"],
        value_name=params["value_name"]
    )
    lista_df.append(df_transformado)

# Concatenar todas en una sola tabla maestra
df_maestro = pd.concat(lista_df, ignore_index=True)

In [0]:
# Remove rows where 'Anio' is not a valid integer
df_maestro = df_maestro[df_maestro['Anio'].apply(lambda x: str(x).isdigit())]

# Optionally, convert 'Anio' to integer type
df_maestro['Anio'] = df_maestro['Anio'].astype(int)

display(df_maestro)

Departamento,Mes,Anio,Weekday,accidentes_totales
El Progreso,null,2020,null,124.0
Sacatepéquez,null,2020,null,202.0
Chimaltenango,null,2020,null,229.0
Escuintla,null,2020,null,440.0
Santa Rosa,null,2020,null,197.0
Sololá,null,2020,null,81.0
Totonicapán,null,2020,null,49.0
Quetzaltenango,null,2020,null,197.0
Suchitepéquez,null,2020,null,201.0
Retalhuleu,null,2020,null,203.0


## 2. Reformateo de hojas

In [0]:
target_cols = ["año", "mes", "día_semana", "departamento", "zona", "tipo_accidente", "accidentes_totales"]

def normalizar_hoja(sheet_name, category):

    df = pd.read_excel(file, sheet_name=sheet_name)
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    
    # Crear columnas maestras si faltan
    for col in target_cols:
        if col not in df.columns:
            df[col] = None
    
    df["origen"] = sheet_name   # opcional: saber de qué cuadro salió
    df["categoria"] = category  # hechos / vehiculos / victimas
    return df[target_cols + ["origen", "categoria"]]

# Detectar todas las hojas
xls = pd.ExcelFile(file)

# Dividir rangos (según índice que revisamos)
hechos_sheets = [f"cuadro {i}" for i in range(1, 17)]
vehiculos_sheets = [f"cuadro {i}" for i in range(17, 31)]
victimas_sheets = [f"cuadro {i}" for i in range(31, 66)]

# Procesar en bloques
hechos = pd.concat([normalizar_hoja(s, "hechos") for s in hechos_sheets], ignore_index=True)
vehiculos = pd.concat([normalizar_hoja(s, "vehiculos") for s in vehiculos_sheets], ignore_index=True)
victimas = pd.concat([normalizar_hoja(s, "victimas") for s in victimas_sheets], ignore_index=True)

In [0]:
# Hoja 1: Total de accidentes por año y departamento
hoja1 = pd.read_excel(file, sheet_name = 'cuadro 1', header= fila_indice1 + 1)

# Reformateo de hoja 1: total de accidentes por año y departamento
hoja1 = hoja1.iloc[1:22,0:6]
hoja1.rename(columns = {'Unnamed: 0':'Departamento'}, inplace=True)
hoja1 = hoja1.melt(
    id_vars=["Departamento"],          # columnas que se mantienen fijas
    var_name="Año",  # nombre de la nueva columna que guarda los encabezados originales
    value_name="accidentes_totales"  # nombre de la columna con los valores
)

hoja1.insert(
    loc=hoja1.columns.get_loc("Año") + 1,  # se vuelve a calcular la posición después de Año
    column="Weekday",
    value=np.nan
)

# Agregar columnas mes y weekday después de Año
hoja1.insert(
    loc=hoja1.columns.get_loc("Año") + 1,  # posición después de Año
    column="Mes",
    value=np.nan
)

hoja1

In [0]:
# Hoja 2: Total de accidentes por depaertamento y por mes
hoja2 = pd.read_excel(file, sheet_name = 'cuadro 2', header= fila_indice1 + 1)
hoja2 = hoja2.iloc[1:22, 0:14].drop(hoja2.columns[1], axis=1)
hoja2.rename(columns = {'Unnamed: 0':'Departamento'}, inplace=True)

hoja2 = hoja2.melt(
    id_vars=["Departamento"],          # columnas que se mantienen fijas
    var_name="Mes",  # nombre de la nueva columna que guarda los encabezados originales
    value_name="accidentes_totales"  # nombre de la columna con los valores
)

# Agregar columnas Año y Weekday para igualar la estructura
hoja2.insert(
    loc=hoja2.columns.get_loc("Departamento"), 
    column="Weekday",
    value=np.nan
)

hoja2.insert(
    loc=hoja2.columns.get_loc("Mes"), 
    column="Año",
    value=np.nan
)
hoja2

In [0]:
# Hoja 3: Total de accidentes por depaertamento y por día de la semana
hoja3 = pd.read_excel(file, sheet_name = 'cuadro 3', header= fila_indice1 + 1)
hoja3 = hoja3.iloc[1:22, 0:9].drop(hoja3.columns[1], axis=1)
hoja3.rename(columns = {'Unnamed: 0':'Departamento'}, inplace=True)

hoja3 = hoja3.melt(
    id_vars=["Departamento"],          # columnas que se mantienen fijas
    var_name="Weekday",  # nombre de la nueva columna que guarda los encabezados originales
    value_name="accidentes_totales"  # nombre de la columna con los valores
)
hoja3

In [0]:
# Hoja 4: accidentes totales por mes y año
hoja4 = pd.read_excel(file, sheet_name = 'cuadro 4', header= fila_indice1 + 1)
hoja4 = hoja4.iloc[1:12, 0:6]
hoja4.rename(columns = {'Unnamed: 0':'Mes'}, inplace=True)

hoja4 = hoja4.melt(
    id_vars=["Mes"],          # columnas que se mantienen fijas
    var_name="Año",  # nombre de la nueva columna que guarda los encabezados originales
    value_name="accidentes_totales"  # nombre de la columna con los valores
)
hoja4.head()

In [0]:
# Hoja 5: 
hoja5 = pd.read_excel(file, sheet_name = 'cuadro 5', header= fila_indice1 + 1)
hoja5 = hoja5.iloc[1:12, 0:9].drop(hoja5.columns[1], axis=1)
hoja5.rename(columns = {'Unnamed: 0':'Mes'}, inplace=True)

hoja5 = hoja5.melt(
    id_vars=["Mes"],          # columnas que se mantienen fijas
    var_name="Weekday",  # nombre de la nueva columna que guarda los encabezados originales
    value_name="accidentes_totales"  # nombre de la columna con los valores
)
hoja5

In [0]:
# Hojas sobre hechos 2020-2024
fila_indice1 = 8 # revisada desde el excel
hoja1_totalacc_anio_dep = pd.read_excel(file, sheet_name = 'cuadro 1', header= fila_indice1 + 1)
hoja4_totalacc_anio_mes = pd.read_excel(file, sheet_name = 'cuadro 4', header= fila_indice1 + 1)
hoja6_totalacc_anio_weekday = pd.read_excel(file, sheet_name = 'cuadro 6', header= fila_indice1 + 1)

# Hojas sobre vehículos 2020-2024
hoja17 = pd.read_excel(file, sheet_name = 'cuadro 17', header= fila_indice1 + 1)
hoja17
